In [ ]:
import pickle
pickle_out= open("./model_trained.p","wb")
pickle.dump(model,pickle_out)
pickle_out.close()

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import pickle
import matplotlib.pyplot as plt

def zoom(img, zoom_factor=3):
    return cv2.resize(img, None, fx=zoom_factor, fy=zoom_factor)

img = cv2.imread("/content/gdrive/MyDrive/IITD/DigitRecognition/C2.tif")
pickle_in= open("/content/gdrive/MyDrive/IITD/DigitRecognition/model_trained.p","rb")
model= pickle.load(pickle_in)

print(model.summary())


img=cv2.resize(img,(1000,1000))

grey = cv2.cvtColor(img.copy(), cv2.COLOR_BGR2GRAY)


kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
grey = cv2.filter2D(src=grey, ddepth=-1, kernel=kernel)


ret, thresh = cv2.threshold(grey, 156, 255, cv2.THRESH_BINARY)

cv2_imshow(thresh)

final_digits = []
centers=[]
rec=[]

while True:
  r = cv2.selectROI("select the area", thresh)
  cropped= thresh[int(r[1]):int(r[1]+r[3]),
                        int(r[0]):int(r[0]+r[2])]
  cv2_imshow(cropped_image)

  cropped= np.asarray(cropped_image)
  cropped= cv2.resize(cropped_image,(32,32))
  cropped=cropped_image/255
  cropped= cropped_image.reshape(1,32,32,1)
  contours,_ = cv2.findContours(cropped, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

  X=[]

  for c in contours:
      x,y,w,h = cv2.boundingRect(c)
      X.append((x,y))
      area= cv2.contourArea(c)
      if area > 15:

          cv2.rectangle(img, (x,y), (x+w, y+h), color=(0, 255, 0), thickness=2)
          digit = thresh[y:y+h, x:x+w]
          resized_digit = cv2.resize(digit, (18,18))
          padded_digit = np.pad(resized_digit, ((5,5),(5,5)), "constant", constant_values=0)
          final_digits.append(padded_digit)

  plt.imshow(img, cmap="gray")
  plt.show()

  zoom= zoom(img)
  cv2.imshow("zoom",zoom)
  for i in range(len(final_digits)):

    f_img= final_digits[i].reshape(1,28,28,1)
    prediction= model.predict(f_img)

    cv2.putText(img, f"{prediction}", (X[i][1]-1,X[i][0]-1), cv2.FONT_HERSHEY_SIMPLEX, 1,
                 (0,255,0), 2, cv2.LINE_AA)


  k=cv2.waitKey(0)
  if k==27:

    cv2.destroyAllWindows()
'''
cv2.waitKey(0)
